## Previous configuration

In [ ]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import warnings
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from skimage.feature import hog
from sklearn.decomposition import PCA
from collections import Counter
import matplotlib.colors as mcolors
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
import random
warnings.filterwarnings("ignore")

## Load data

In [ ]:
def load_dataset(dataset_path):
    dataset = []
    num_folders = sorted(os.listdir(dataset_path), key=lambda x: int(x))
    
    for num_folder in num_folders:
        num_folder_path = os.path.join(dataset_path, num_folder)
        for img_name in os.listdir(num_folder_path):
            img_path = os.path.join(num_folder_path, img_name)
            img = cv2.imread(img_path)
            flipped = tf.image.flip_left_right(img)
            flipped = np.array(flipped)
            flipped = cv2.cvtColor(flipped, cv2.COLOR_RGB2GRAY)
            img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
            
            random_value = random.uniform(1.2, 1.5)
            increase = np.ones((64, 64)) * random_value
            random_value = random.uniform(0.5, 0.8)
            decrease = np.ones((64, 64)) * 0.65
            
            img = img/255.0
            dataset.append((img, num_folder))
           
            flipped = flipped/255.0
            dataset.append((flipped, num_folder))
            
            bright_img = np.clip(img * increase, 0, 1)
            dataset.append((bright_img, num_folder))
            
            bright_flipped = np.clip(flipped * increase, 0, 1)
            dataset.append((bright_flipped, num_folder))
            
            bright_img2 = np.clip(img * decrease, 0, 1)
            dataset.append((bright_img2, num_folder))
            
            bright_flipped2 = np.clip(flipped * decrease, 0, 1)
            dataset.append((bright_flipped2, num_folder))
            
            random_value = random.uniform(0.02, 0.033)
            noise_img1 = img + np.random.normal(0, random_value, img.shape)
            noise_img1 = np.clip(noise_img1, 0, 1)
            dataset.append((noise_img1, num_folder))

            noise_img2 = flipped + np.random.normal(0, random_value, flipped.shape)
            noise_img2 = np.clip(noise_img2, 0, 1)
            dataset.append((noise_img2, num_folder))
            
            blur_img1 = cv2.GaussianBlur(img, (5, 5), 0)
            dataset.append((blur_img1, num_folder))
            blur_img2 = cv2.GaussianBlur(flipped, (5, 5), 0)
            dataset.append((blur_img2, num_folder))
           
    return dataset
data=load_dataset("dataset")
noms = {}
noms['41']="Serrano"
noms['42']="Cadevall"
noms['43']="Blánquez"

## New Images

In [ ]:
def get_next_image_name(folder_path):
    # Find the highest numbered image file in the dataset folder
    existing_images = [f for f in os.listdir(folder_path) if f.startswith("img") and f.endswith(".png")]
    if not existing_images:
        return "img0.png"  # If no images found, start with img0.jpg
    else:
        highest_number = max([int(img.split("img")[1].split(".png")[0]) for img in existing_images])
        next_number = highest_number + 1
        return "img{}.png".format(next_number)

def capture_img(folder_number,save=True):
    
    dataset_folder = "dataset"
    
    cap = cv2.VideoCapture(0)
    ret, frame = cap.read()

    grayscale_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(grayscale_frame, scaleFactor=1.1, minNeighbors=5)
    if len(faces) > 0:
        (x, y, w, h) = faces[0]  # Assuming only one face is detected, you can modify this if needed
        face_image = grayscale_frame[y:y+h, x:x+w]
        grayscale_frame = cv2.resize(face_image, (64, 64))
        
        if save:
            folder_path = os.path.join(dataset_folder, str(folder_number))
            # Check if the folder exists, if not, create it
            if not os.path.exists(folder_path):
                os.makedirs(folder_path)
            # Save the grayscale image in the dataset folder
            next_img_name = get_next_image_name(folder_path)  # You need to define get_next_image_name() function
            img_path = os.path.join(folder_path, next_img_name)
            cv2.imwrite(img_path, grayscale_frame)
            
        # Release the webcam
        cap.release()
        
        print("Image captured, resized to 64x64 pixels, converted to grayscale, and normalized")
        
        return grayscale_frame
    else:
        print("No face detected")
        cap.release()
        return None


plt.imshow(capture_img(41),cmap="gray")
plt.show()

## Case of use

In [ ]:
import numpy as np
import random
import os
import cv2
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from skimage.feature import hog
from time import sleep

def identify():
    dataset_path = "dataset"
    data = load_dataset(dataset_path)
    print("Wait")
    X_data=[]
    for img in data:
        X_data.append(img[0])
    X_data=np.array(X_data)
    y_data=[]
    for target in data:
        y_data.append(target[1])
    y_data=np.array(y_data)
    
    X_hog=[]
    for img in X_data:
        fd, hog_image = hog(img, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(1, 1), visualize=True, multichannel=False)
        X_hog.append(hog_image)
    X_hog=np.array(X_hog)

    # Reshape images
    n_samples, height, width = X_hog.shape
    X_hog = np.reshape(X_hog, (n_samples, height * width))

    # Apply PCA
    pca = PCA()  # Choose number of components as per requirement
    pca.fit(X_hog)
    
    component=500
    
    eigenfaces = pca.components_[:component]

    threshold = 0.3
    
    weights = eigenfaces @ (X_hog - pca.mean_).T
    print("3")
    sleep(1)
    print("2")
    sleep(1)
    print("1")
    sleep(1)
    cap = cv2.VideoCapture(0)
    ret, frame = cap.read()
    grayscale_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)   
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(grayscale_frame, scaleFactor=1.1, minNeighbors=5)
    
    if len(faces) > 0:
        (x, y, w, h) = faces[0]  # Assuming only one face is detected, you can modify this if needed
        face_image = grayscale_frame[y:y+h, x:x+w]
        grayscale_frame = cv2.resize(face_image, (64, 64))
        plt.imshow(grayscale_frame,cmap="gray")
        plt.show()
        fd, hog_target = hog(grayscale_frame, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(1, 1), visualize=True, multichannel=False)
        hog_target = hog_target/255
        img_weight = eigenfaces @ (hog_target.reshape(1, -1) - pca.mean_).T

        euclidean_distance = np.linalg.norm(weights - img_weight, axis=0)
        closest_index = np.argsort(euclidean_distance)[0]
        print(euclidean_distance[closest_index])
        if euclidean_distance[closest_index] < threshold:
            neighbor_label = y_data[closest_index]
            plt.imshow(X_data[closest_index],cmap="gray")
            plt.show()

            print("Predicted class:", neighbor_label)
            if (neighbor_label in ["41","42","43"]):
                print(noms[neighbor_label])
        else:
            print("Not in our dataset")
    else:
        print("No face detected.")

    cap.release()

identify()